In [24]:
# Dependencies
import pandas as pd
import requests
import psycopg2
from sqlalchemy import create_engine, table

In [4]:
# Importing recipes from spoonacular
api_key = "a21364208c9d43719315d65c34faa384"  ### will need to grab from config file
# Using url for "Get Recipe Information" JSONs: https://spoonacular.com/food-api/docs#Get-Recipe-Information
# url = "https://api.spoonacular.com/recipes/10/information?includeNutrition=false"

# id_df = pd.read_csv('unique_recipes.csv')
# id = id_df['0']

id = range(10)

# Initializing lists for url and query_url
url = []
query_url = [None]*len(id)

# Loop through a set of recipes up to a pre-determined amount
for i in range(0, len(id)):
    url.append(f"https://api.spoonacular.com/recipes/{id[i]}/information?includeNutrition=false")
    query_url[i] = f'{url[i]}&apiKey={api_key}'


In [5]:
from pprint import pprint
# Initializing lists for results from query
recipe_id = []
recipe_name = []
servings = []
diets = []
dairyFree = []
vegan = []
vegetarian = []
ketogenic = []
images = []
source_url = []
ingredients = []
individual_ingre=[]

# Looping through recipe JSONs
count = 0
ingred_list = []

### Change the range later after number of recipes is agreed on.
for i in range(0, len(id)):
    
    try:
        random_list=[]
        response = requests.get(query_url[i]).json()
    
        recipe_id.append(response['id'])
        recipe_name.append(response['title'])
        servings.append(response['servings'])
        diets.append(response['diets'])
        dairyFree.append(response['dairyFree'])
        vegan.append(response['vegan'])
        vegetarian.append(response['vegetarian'])
#         ketogenic.append(response['ketogenic'])
        images.append(response['image'])
        source_url.append(response['sourceUrl'])
        ingredients.append(response['extendedIngredients'])
        
        for ingre in ingredients:
            for i in range(len(ingre)):
                random_list.append(ingre[i]["name"])
        individual_ingre.append(random_list)
    except:
        print(f'Recipe {id[i]} was not found.')

Recipe 0 was not found.
Recipe 8 was not found.


In [15]:
# Develop dataframe from queried recipes
recipe_df = pd.DataFrame({'ID': recipe_id,
                          'Recipe Name': recipe_name,
                          'Servings': servings,
                          'Diets': diets,
                          'Dairy Free': dairyFree,
                          'Vegan': vegan,
                          'Vegetarian': vegetarian,
                          'Image URL': images, 
                          'Source URL': source_url,
                          'Ingredients': individual_ingre
                        })
                        
recipe_forsql_df = recipe_df.rename(columns = {'Recipe Name':'Recipe_Name',
                                               'Dairy Free':'Dairy_Free',
                                               'Image URL':'Image_URL',
                                               'Source URL':'Source_URL'
                                              })
# Display recipe_df
recipe_df.head()

,ID,Recipe Name,Servings,Diets,Dairy Free,Vegan,Vegetarian,Image URL,Source URL,Ingredients
0,1,Fried Anchovies with Sage,3,"[dairy free, pescatarian]",True,False,False,https://spoonacular.com/recipeImages/1-556x370...,http://latavolamarcherecipebox.blogspot.com/20...,"[anchovies, baking powder, egg, flour, sage le..."
1,2,Anchovies Appetizer With Breadcrumbs & Scallions,8,"[dairy free, pescatarian]",True,False,False,https://spoonacular.com/recipeImages/2-556x370...,http://www.thekitchn.com/other-two-veg-recipes...,"[anchovies, baking powder, egg, flour, sage le..."
2,3,"Carrots, Cauliflower And Anchovies",1,"[dairy free, pescatarian]",True,False,False,https://spoonacular.com/recipeImages/3-556x370...,http://saladpride.blogspot.com/2011/06/carrots...,"[anchovies, baking powder, egg, flour, sage le..."
3,4,Bap Story: Stir Fried Anchovies (Myulchi Bokkeum),2,"[gluten free, dairy free]",True,False,False,https://spoonacular.com/recipeImages/4-556x370...,http://bapstory.blogspot.com/2011/08/stir-frie...,"[anchovies, baking powder, egg, flour, sage le..."
4,5,"Bread, Butter And Anchovies",1,[pescatarian],False,False,False,https://spoonacular.com/recipeImages/5-556x370...,http://en.julskitchen.com/tuscany/grandma-menn...,"[anchovies, baking powder, egg, flour, sage le..."


In [27]:
rds_connection_string = 'postgres:postgres@localhost:5432/Project_2'
engine = create_engine(f'postgresql://{rds_connection_string}')

recipe_df.to_sql('recipe_df', con=engine, index=False, if_exists='replace')